In [ ]:
# Install required packages
!pip install transformers datasets torch accelerate peft ctransformers --upgrade
!pip install bitsandbytes  # For 8-bit quantization support
!pip install sentencepiece  # For DeepSeek tokenizer

In [ ]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, TrainerCallback
from datasets import Dataset
from peft import LoraConfig, get_peft_model
import logging
import os

In [ ]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('training_log.txt'),
        logging.StreamHandler()
    ]
)

# Custom callback for loss logging
class LossLoggingCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero and logs is not None:
            if 'loss' in logs:
                epoch = state.epoch
                step = state.global_step
                loss = logs['loss']
                logging.info(f'Epoch: {epoch:.2f}, Step: {step}, Loss: {loss:.4f}')
            if 'eval_loss' in logs:
                eval_loss = logs['eval_loss']
                logging.info(f'Evaluation Loss: {eval_loss:.4f}')

In [ ]:
# Load and process the JSONL data
def load_conversations(file_path):
    conversations = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            messages = data['messages']
            # Format conversation
            conv = ""
            for msg in messages:
                role = msg['role']
                content = msg['content'] 
                conv += f"<{role}>{content}</s>"
            conversations.append({'conversation': conv})
    return conversations

In [ ]:
# Load training data
train_data = load_conversations('output_30.jsonl')
val_data = load_conversations('output_70.jsonl')

# Create datasets
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

In [ ]:
# Initialize model and tokenizer
model_name = "deepseek-ai/DeepSeek-R1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# LoRA configuration - adjusted for DeepSeek
lora_config = LoraConfig(
    r=16,  # Increased rank for better adaptation
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # DeepSeek attention modules
    lora_dropout=0.1,
)

# Create PEFT model
model = get_peft_model(model, lora_config)

In [ ]:
# Training arguments with added logging
training_args = TrainingArguments(
    output_dir="tiuz-chat-model",
    num_train_epochs=5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=1e-4,
    save_strategy="epoch",  # Changed to save per epoch
    save_total_limit=1,      # Only keep latest checkpoint
    evaluation_strategy="epoch",  # Evaluate per epoch
    bf16=True,
    gradient_checkpointing=True,
    optim='adamw_torch',
    load_best_model_at_end=True,
    save_only_model=True,    # Don't save optimizer states
    overwrite_output_dir=True, # Overwrite existing checkpoints
    logging_dir='./logs',      # Directory for tensorboard logs
    logging_strategy="steps",  # Log every N steps
    logging_steps=10,          # Log every 10 steps
    logging_first_step=True,   # Log the first step
    report_to=["tensorboard"], # Use tensorboard for logging
)

# Create trainer with callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[LossLoggingCallback()]
)

In [ ]:
# Start training
trainer.train()

# Save the model
trainer.save_model("tiuz-chat-final")

# Convert to GGUF format
!python3 -m ctransformers.lib.converting tiuz-chat-final tiuz-chat.gguf